# Enrich the cannibalization events

In [1]:
# Step 1: Set environment
# We need to set environment's correct values if we want to load project modules. 
import sys, os

# It's important load the PROJECT_ROOT path. We need to replace PROJECT_ROOT with correct folder
# Module is placed in api folder inside project root folder
# PROJECT_ROOT = '/Users/44371/Documents/Cases/V7FC/bain-vantage'
PROJECT_ROOT = '/home/jacrisol/github/bain-vantage'
MODULE_FULL_PATH = os.path.join(PROJECT_ROOT, 'api')
sys.path.insert(1, MODULE_FULL_PATH)


from api.settings import env, config

# config file sample can be foun at PROJECT_ROOT/api/config.env.sample. All keys and passwods are placed there.
# in this step, we load this file to get all carto api keys needed.
# env.read_env('/Users/44371/Documents/Cases/V7FC/bain-vantage/api/notebooks.env')
env.read_env('/home/jacrisol/github/bain-vantage/api/notebooks.env')
with env.prefixed("BAIN_VANT_API_"):
    config['carto'] = {
        'base_url': env.str('CARTO_BASE_URL', ''),
        'user': env.str('CARTO_ADMIN_USER', 'xxx'),
        'api_key': env.str('CARTO_ADMIN_API_KEY', 'xxx'),
    }
    
# Step 2: create carto variable
# create carto object to be able to use methods inside it. We import all necessary classes for the samples too 

from etl.cf_model import CartoFramesModel, GeometryType
from etl.constants.global_constants import meters_in_mile
from cartoframes import to_carto, create_table_from_query, read_carto

carto = CartoFramesModel()

/home/jacrisol/.virtualenvs/vantage/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
urb_buffers = {'Urban': 1, 'Sub-urban': 3, 'Rural': 5, 'Other': 5}
seed_table = 'vtg_test_ws_enrichment'
cann_candidates = 'vtg_test_ws_cann_candidate'
site_table = 'vtg_customer_locations'

In [23]:
def enrich_seeds():
    seed_base = carto.get_geopandas_from_query(f"select * from {seed_table} where urbanicity in ('Urban', 'Sub-urban', 'Rural')")
    df = carto.make_buffer(seed_table, urb_buffers, 'urbanicity')
    # Drop Null geoms
    df.dropna(subset=['the_geom'], inplace=True)
    seed_enriched = carto.enrichment_variables(df, ['total_pop_3cf008b3'])
    
    seed_base = seed_base.merge(seed_enriched[['cartodb_id', 'total_pop']],
                            on='cartodb_id', how='left')
    return seed_base

In [34]:
def enrich_sites():

    sites = read_carto(site_table)
    sites_zipcode = carto.enrichment_variables(sites, ['geoid_c90eb55a'], GeometryType.POINTS)
    sites_zipcode['geoid'] = sites_zipcode['geoid'].fillna(0).astype('int')
    sites_zipcode.rename(columns={'geoid':'zipcode'}, inplace=True)
    
    to_carto(sites_zipcode, 'vtg_test_ws_customer_location_zipcode', if_exists='replace')
        
    # enrich with urbanicity data
    sql = f"""
        WITH updated_sites AS(
            SELECT
               *
            FROM
                vtg_test_ws_customer_location_zipcode)
        SELECT
            a.*
            , b.city_name
            , b.cbsa_code
            , b.cbsa_name
            , b.dma_code
            , b.dma_name
            , b.state_fip
            , b.state_abb
            , b.region
            , b.urbanicity
        FROM
            updated_sites a
        LEFT JOIN
            btunnell9.zip_urbanicity b
        ON
            LPAD(CAST(a.zipcode as VARCHAR), 5, '0') = b.zipcode
        """

    sites_enriched = carto.get_geopandas_from_query(sql)
    to_carto(sites_enriched, 'vtg_test_ws_customer_location_enrichment', if_exists='replace')
    
    
    # Create buffers to enrich with total_pop
    print("creating buffers...")
    sites_buffer = carto.make_buffer('vtg_test_ws_customer_location_enrichment', urb_buffers, 'urbanicity')
    print("enrichment population...")
    # Drop Null geoms
    sites_buffer.dropna(subset=['the_geom'], inplace=True)
    sites_buffer = carto.enrichment_variables(sites_buffer, ['total_pop_3cf008b3'])
    
    merge = sites_enriched.merge(sites_buffer[['cartodb_id', 'total_pop']], on='cartodb_id', how='left')

    

    return merge


In [56]:
def enrich_cann_event():
    #Calculating distance
    sql = f"""
        SELECT
            m.store_id
            , m.seed_id
            , st_distance(geography(c.the_geom), geography(s.the_geom))/{meters_in_mile} distancemiles,
            0.0 as pred_can
        FROM
            {cann_candidates} m
        LEFT JOIN
            {site_table} c
            ON m.store_id = c.store_id
        LEFT JOIN
            {seed_table} s
            ON m.seed_id = s.seed_id
    """
    distance_tbl = carto.get_geopandas_from_query(sql)
    return distance_tbl

## Calculating cannibalizations events candidates

from vtg_test_ws_cann_candidate we generated in step 4, we enrich all necessary data and save again in vtg_test_ws_cann_candidate

In [57]:
def merge_data(events, seeds, sites):
    
    can_tbl = events.copy()

    # Seed urbanicity and population
    can_tbl = can_tbl.merge(seeds[['seed_id', 'urbanicity', 'cartodb_id', 'total_pop']], on='seed_id', how='left')

    # Clean and rename columns
    can_tbl.drop(columns=['cartodb_id'], inplace=True)
    can_tbl.rename(columns={'urbanicity':'n_urbanicity', 'total_pop':'n_popcy'}, inplace=True)

    # Site urbanicity and pop
    can_tbl = can_tbl.merge(sites[['store_id', 'total_pop', 'urbanicity']],
                            on='store_id', how='left')

    # Clean and rename columns
    can_tbl.rename(columns={'urbanicity':'o_urbanicity', 'total_pop':'o_popcy'}, inplace=True)
    
    return can_tbl

In [58]:
def bain_method():
    print('Enrich seeds...')
    seeds = enrich_seeds()
    
    print('Enrich sites...')
    sites = enrich_sites()
    
    events = enrich_cann_event()

    print('Merging data..')
    cann_enrich = merge_data(events, seeds, sites)
    
    
    to_carto(cann_enrich, cann_candidates, if_exists='replace')
    print('Cann enrichment finished !')

In [59]:
bain_method()

Enrich seeds...
Enrich sites...


[2021-02-05T18:16:30Z] (272240) {carto.py:130} INFO - Success! Data uploaded to table "vtg_test_ws_customer_location_zipcode" correctly
[2021-02-05T18:16:39Z] (272240) {carto.py:130} INFO - Success! Data uploaded to table "vtg_test_ws_customer_location_enrichment" correctly


creating buffers...
enrichment population...
Merging data..


[2021-02-05T18:17:27Z] (272240) {carto.py:130} INFO - Success! Data uploaded to table "vtg_test_ws_cann_candidate" correctly


Cann enrichment finished !
